In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

# Configuration Parameters
window_size = 800
step_size = 50
min_length = 500
d = 2
step = 1

idm_param_names = ['vmax', 'dsafe', 'tsafe', 'amax', 'amin', 's_a', 's_v']

def find_sliding_window_segments(track, window_size=100, step_size=50, min_length=50):
    segments = []
    n = len(track['vFollReal'])
    
    for start in range(0, n - window_size + 1, step_size):
        end = start + window_size - 1
        
        window_data = track['vFollReal'][start:end+1]
        valid_count = np.sum(~np.isnan(window_data))
        
        if valid_count >= min_length:
            seg = {
                'start': start, 
                'end': end, 
                'type': f'window_{len(segments)}',
                'window_id': len(segments)
            }
            segments.append(seg)
    
    return segments

def build_full_multi_vehicle_data(tracks):
    vt_list = []
    s_list = []
    dv_list = []
    label_v_list = []
    id_idx_list = []
    vehicle_info = {}
    
    vehicle_counter = 0
    
    for vehicle_id, track in tracks.items():
        vt_veh = track['vFollReal']
        s_veh = track['sReal']
        dv_veh = track['dvReal']
        label_v_veh = track['vFollReal_next']
        
        mask = (~np.isnan(vt_veh)) & (~np.isnan(s_veh)) & (~np.isnan(dv_veh)) & (~np.isnan(label_v_veh))
        valid_count = np.sum(mask)
        
        if valid_count >= min_length:
            vt_list.extend(vt_veh[mask])
            s_list.extend(s_veh[mask])
            dv_list.extend(dv_veh[mask])
            label_v_list.extend(label_v_veh[mask])
            id_idx_list.extend([vehicle_counter] * valid_count)
            
            vehicle_info[vehicle_id] = {
                'data_index': vehicle_counter,
                'n_points': valid_count,
                'original_indices': np.where(mask)[0]
            }
            vehicle_counter += 1
    
    if vehicle_counter == 0:
        return {'n_vehicles': 0}
    
    return {
        'vt': np.array(vt_list),
        's': np.array(s_list),
        'dv': np.array(dv_list),
        'label_v': np.array(label_v_list),
        'id_idx': np.array(id_idx_list),
        'n_vehicles': vehicle_counter,
        'vehicle_info': vehicle_info
    }

def build_multi_vehicle_data_for_window(tracks, window_info):
    vt_list = []
    s_list = []
    dv_list = []
    label_v_list = []
    id_idx_list = []
    vehicle_info = {}
    
    start, end = window_info['start'], window_info['end']
    vehicle_counter = 0
    
    for vehicle_id, track in tracks.items():
        vt_veh = track['vFollReal'][start:end+1]
        s_veh = track['sReal'][start:end+1]
        dv_veh = track['dvReal'][start:end+1]
        label_v_veh = track['vFollReal_next'][start:end+1]
        
        mask = (~np.isnan(vt_veh)) & (~np.isnan(s_veh)) & (~np.isnan(dv_veh)) & (~np.isnan(label_v_veh))
        valid_count = np.sum(mask)
        
        if valid_count >= min_length:
            vt_list.extend(vt_veh[mask])
            s_list.extend(s_veh[mask])
            dv_list.extend(dv_veh[mask])
            label_v_list.extend(label_v_veh[mask])
            id_idx_list.extend([vehicle_counter] * valid_count)
            
            vehicle_info[vehicle_id] = {
                'data_index': vehicle_counter,
                'n_points': valid_count
            }
            vehicle_counter += 1
    
    if vehicle_counter == 0:
        return {'n_vehicles': 0}
    
    return {
        'vt': np.array(vt_list),
        's': np.array(s_list),
        'dv': np.array(dv_list),
        'label_v': np.array(label_v_list),
        'id_idx': np.array(id_idx_list),
        'n_vehicles': vehicle_counter,
        'vehicle_info': vehicle_info
    }

def extract_ar_coefficients_for_vehicle(trace, vehicle_id, d, multi_vehicle_data):
    if 'rho' not in trace.posterior:
        return None, None
    
    vehicle_info = multi_vehicle_data['vehicle_info'].get(vehicle_id)
    if vehicle_info is None:
        return None, None
    
    vehicle_index = vehicle_info['data_index']
    
    rho_data = trace.posterior['rho'].values
    
    if vehicle_index >= rho_data.shape[2]:
        return None, None
    
    ar_means = []
    ar_stds = []
    
    for lag in range(d):
        lag_data = rho_data[:, :, vehicle_index, lag].flatten()
        ar_means.append(np.mean(lag_data))
        ar_stds.append(np.std(lag_data))
    
    return ar_means, ar_stds

def extract_idm_parameters_for_vehicle(trace, vehicle_id, multi_vehicle_data):
    param_means = {}
    param_stds = {}
    
    vehicle_info = multi_vehicle_data['vehicle_info'].get(vehicle_id)
    if vehicle_info is None:
        return None, None
    
    vehicle_index = vehicle_info['data_index']
    
    for param_base in ['vmax', 'dsafe', 'tsafe', 'amax', 'amin']:
        param_name = f"{param_base}_{vehicle_index}"
        if param_name in trace.posterior:
            param_data = trace.posterior[param_name].values.flatten()
            param_means[param_base] = np.mean(param_data)
            param_stds[param_base] = np.std(param_data)
    
    for param_base in ['s_a', 's_v']:
        param_name = f"{param_base}_{vehicle_index}"
        if param_name in trace.posterior:
            param_data = trace.posterior[param_name].values.flatten()
            param_means[param_base] = np.mean(param_data)
            param_stds[param_base] = np.std(param_data)
    
    return param_means, param_stds

def extract_population_parameters(trace, d):
    population_params = {}
    
    if 'rho_mu' in trace.posterior:
        rho_mu_data = trace.posterior['rho_mu'].values
        population_params['rho_mu'] = {}
        for lag in range(d):
            lag_data = rho_mu_data[:, :, lag].flatten()
            population_params['rho_mu'][f'lag_{lag+1}'] = {
                'mean': np.mean(lag_data),
                'std': np.std(lag_data)
            }
    
    population_idm_mappings = {
        'log_mu_vmax': 'mu_vmax',
        'log_mu_dsafe': 'mu_dsafe', 
        'log_mu_tsafe': 'mu_tsafe',
        'log_mu_amax': 'mu_amax',
        'ratio_mu': 'ratio_mu'
    }
    
    population_params['idm'] = {}
    for trace_name, display_name in population_idm_mappings.items():
        if trace_name in trace.posterior:
            param_data = trace.posterior[trace_name].values.flatten()
            
            if trace_name.startswith('log_'):
                exp_data = np.exp(param_data)
                population_params['idm'][display_name] = {
                    'mean': np.mean(exp_data),
                    'std': np.std(exp_data),
                    'log_mean': np.mean(param_data),
                    'log_std': np.std(param_data)
                }
            else:
                population_params['idm'][display_name] = {
                    'mean': np.mean(param_data),
                    'std': np.std(param_data)
                }
    
    scaling_factors = {
        'mu_vmax': 25.0,
        'mu_dsafe': 2.0,
        'mu_tsafe': 1.6,
        'mu_amax': 1.5,
        'ratio_mu': 1.0
    }
    
    population_params['idm_scaled'] = {}
    for param_name, param_info in population_params['idm'].items():
        if param_name in scaling_factors:
            scaling_factor = scaling_factors[param_name]
            scaled_mean = param_info['mean'] * scaling_factor
            scaled_std = param_info['std'] * scaling_factor
            
            population_params['idm_scaled'][param_name] = {
                'scaled_mean': scaled_mean,
                'scaled_std': scaled_std,
                'raw_mean': param_info['mean'],
                'raw_std': param_info['std'],
                'scaling_factor': scaling_factor
            }
    
    if 'chol' in trace.posterior:
        chol_data = trace.posterior['chol'].values
        population_params['chol_covariance'] = {
            'mean': np.mean(chol_data, axis=(0, 1)),
            'std': np.std(chol_data, axis=(0, 1))
        }
    
    return population_params

def extract_all_parameters_for_vehicle(trace, vehicle_id, d, multi_vehicle_data):
    ar_means, ar_stds = extract_ar_coefficients_for_vehicle(trace, vehicle_id, d, multi_vehicle_data)
    idm_means, idm_stds = extract_idm_parameters_for_vehicle(trace, vehicle_id, multi_vehicle_data)
    
    return {
        'ar_means': ar_means,
        'ar_stds': ar_stds,
        'idm_means': idm_means,
        'idm_stds': idm_stds
    }

def run_simultaneous_calibration():
    print("Starting simultaneous calibration of all vehicles...")
    
    tracks = ar_idm_data['tracks']
    
    print(f"Total number of vehicles: {len(tracks)}")
    print(f"AR lag (d): {d}")
    
    full_multi_vehicle_data = build_full_multi_vehicle_data(tracks)
    
    if full_multi_vehicle_data['n_vehicles'] == 0:
        print("No valid vehicle data found")
        return None
    
    print(f"Simultaneously calibrating {full_multi_vehicle_data['n_vehicles']} vehicles...")
    
    try:
        full_trace, full_model = train_ar_model(full_multi_vehicle_data, d=d, step=step)
        print("Full-time calibration completed successfully!")
        
        population_params = extract_population_parameters(full_trace, d)
        
        print("\n" + "="*60)
        print("POPULATION PARAMETERS")
        print("="*60)
        
        print("\nPopulation AR Coefficients (rho_mu):")
        print("-" * 40)
        if 'rho_mu' in population_params:
            for lag in range(d):
                lag_key = f'lag_{lag+1}'
                if lag_key in population_params['rho_mu']:
                    lag_info = population_params['rho_mu'][lag_key]
                    print(f"  ρ_mu (lag {lag+1}): {lag_info['mean']:.3f} ± {lag_info['std']:.3f}")
        
        print("\nPopulation IDM Parameters (with scaling factors applied):")
        print("-" * 50)
        if 'idm_scaled' in population_params:
            for param_name, param_info in population_params['idm_scaled'].items():
                print(f"  {param_name}: {param_info['scaled_mean']:.3f} ± {param_info['scaled_std']:.3f} "
                      f"(scaling factor: {param_info['scaling_factor']})")
        
        print("\nPopulation IDM Parameters (raw, log-scale):")
        print("-" * 45)
        if 'idm' in population_params:
            for param_name, param_info in population_params['idm'].items():
                if param_name.startswith('mu_'):
                    if 'log_mean' in param_info:
                        print(f"  {param_name}: exp({param_info['log_mean']:.3f} ± {param_info['log_std']:.3f})")
                    else:
                        print(f"  {param_name}: {param_info['mean']:.3f} ± {param_info['std']:.3f}")
        
        full_time_results = {}
        for vehicle_id, vehicle_info in full_multi_vehicle_data['vehicle_info'].items():
            driver_name = tracks[vehicle_id].get('driver_id', f'Driver_{vehicle_id}')
            vehicle_pair = tracks[vehicle_id].get('vehicle_pair', 'Unknown')
            
            all_params = extract_all_parameters_for_vehicle(full_trace, vehicle_id, d, full_multi_vehicle_data)
            
            if all_params['ar_means'] is not None:
                full_time_results[vehicle_id] = {
                    'driver_name': driver_name,
                    'vehicle_pair': vehicle_pair,
                    'full_time_ar_means': all_params['ar_means'],
                    'full_time_ar_stds': all_params['ar_stds'],
                    'full_time_idm_means': all_params['idm_means'],
                    'full_time_idm_stds': all_params['idm_stds']
                }
                print(f"\n  {driver_name}:")
                print(f"    AR coefficients: {[f'{x:.3f}' for x in all_params['ar_means']]}")
                print(f"    IDM parameters: {all_params['idm_means']}")
        
        return {
            'full_trace': full_trace,
            'full_model': full_model,
            'full_time_results': full_time_results,
            'population_params': population_params,
            'multi_vehicle_data': full_multi_vehicle_data
        }
        
    except Exception as e:
        print(f"Error in full-time calibration: {e}")
        return None

def run_sliding_window_analysis_corrected():
    print("\nStarting corrected sliding window analysis...")
    
    tracks = ar_idm_data['tracks']
    sliding_window_results = {}
    window_population_params = {}
    
    print("Finding common time windows across all vehicles...")
    
    all_vehicle_segments = {}
    for vehicle_id, track in tracks.items():
        segments = find_sliding_window_segments(track, window_size=window_size, step_size=step_size, min_length=min_length)
        all_vehicle_segments[vehicle_id] = segments
        print(f"  {track.get('driver_id', f'Driver_{vehicle_id}')}: {len(segments)} segments")
    
    reference_vehicle = list(all_vehicle_segments.keys())[0]
    common_windows = all_vehicle_segments[reference_vehicle]
    
    print(f"Using {len(common_windows)} common time windows")
    
    for window_idx, window_info in enumerate(common_windows):
        print(f"\nProcessing Window {window_idx} (start: {window_info['start']}, end: {window_info['end']})")
        
        multi_vehicle_window_data = build_multi_vehicle_data_for_window(tracks, window_info)
        
        if multi_vehicle_window_data['n_vehicles'] > 0:
            print(f"  Simultaneously calibrating {multi_vehicle_window_data['n_vehicles']} vehicles...")
            
            try:
                trace, model = train_ar_model(multi_vehicle_window_data, d=d, step=step)
                
                window_population = extract_population_parameters(trace, d)
                window_population_params[window_info['window_id']] = {
                    'population_params': window_population,
                    'start': window_info['start'],
                    'end': window_info['end'],
                    'n_vehicles': multi_vehicle_window_data['n_vehicles']
                }
                
                print(f"  Window {window_idx} population parameters extracted")
                
                for vehicle_id, vehicle_info in multi_vehicle_window_data['vehicle_info'].items():
                    if vehicle_id not in sliding_window_results:
                        driver_name = tracks[vehicle_id].get('driver_id', f'Driver_{vehicle_id}')
                        sliding_window_results[vehicle_id] = {
                            'driver_name': driver_name,
                            'vehicle_pair': tracks[vehicle_id].get('vehicle_pair', 'Unknown'),
                            'window_results': []
                        }
                    
                    all_params = extract_all_parameters_for_vehicle(trace, vehicle_id, d, multi_vehicle_window_data)
                    
                    if all_params['ar_means'] is not None:
                        sliding_window_results[vehicle_id]['window_results'].append({
                            'window_id': window_info['window_id'],
                            'start': window_info['start'],
                            'end': window_info['end'],
                            'ar_means': all_params['ar_means'],
                            'ar_stds': all_params['ar_stds'],
                            'idm_means': all_params['idm_means'],
                            'idm_stds': all_params['idm_stds'],
                            'n_obs': vehicle_info['n_points']
                        })
                
                print(f"  Window {window_idx} completed successfully")
                
            except Exception as e:
                print(f"  Error fitting window {window_idx}: {e}")
    
    for vehicle_id in sliding_window_results:
        sliding_window_results[vehicle_id]['window_results'].sort(key=lambda x: x['window_id'])
        print(f"{sliding_window_results[vehicle_id]['driver_name']}: {len(sliding_window_results[vehicle_id]['window_results'])} windows")
    
    print("\n" + "="*60)
    print("SLIDING WINDOW POPULATION PARAMETERS SUMMARY")
    print("="*60)
    
    for window_id, window_info in window_population_params.items():
        print(f"\nWindow {window_id} (vehicles: {window_info['n_vehicles']}):")
        pop_params = window_info['population_params']
        
        if 'rho_mu' in pop_params:
            ar_str = " | ".join([f"ρ_mu_lag{lag+1}: {pop_params['rho_mu'][f'lag_{lag+1}']['mean']:.3f}" 
                               for lag in range(d)])
            print(f"  Population AR: {ar_str}")
        
        if 'idm_scaled' in pop_params:
            idm_str = " | ".join([f"{name}: {info['scaled_mean']:.3f}" 
                                for name, info in pop_params['idm_scaled'].items()])
            print(f"  Population IDM: {idm_str}")
    
    return sliding_window_results, window_population_params

def print_detailed_parameter_analysis(full_time_results, sliding_window_results):
    print("\n" + "="*80)
    print("DETAILED PARAMETER ANALYSIS")
    print("="*80)
    
    full_time_results_dict = full_time_results['full_time_results']
    
    for vehicle_id, full_data in full_time_results_dict.items():
        if vehicle_id not in sliding_window_results:
            continue
        
        window_data = sliding_window_results[vehicle_id]
        driver_name = full_data['driver_name']
        
        print(f"\n{driver_name}:")
        print("="*50)
        
        print("\nAR COEFFICIENTS ANALYSIS:")
        print("-" * 30)
        
        window_ar_means = np.array([result['ar_means'] for result in window_data['window_results']])
        
        for lag in range(d):
            full_mean = full_data['full_time_ar_means'][lag]
            full_std = full_data['full_time_ar_stds'][lag]
            window_mean = np.mean(window_ar_means[:, lag])
            window_std = np.std(window_ar_means[:, lag])
            
            diff = abs(full_mean - window_mean)
            diff_ratio = diff / full_mean if full_mean != 0 else diff
            
            print(f"  ρ (lag {lag+1}):")
            print(f"    Full-Time:  {full_mean:.3f} ± {full_std:.3f}")
            print(f"    Window Avg: {window_mean:.3f} ± {window_std:.3f}")
            print(f"    Difference: {diff:.3f} ({diff_ratio:.1%})")
            
            temporal_variability = np.std(window_ar_means[:, lag])
            print(f"    Temporal Variability: {temporal_variability:.3f}")
        
        print("\nIDM PARAMETERS ANALYSIS:")
        print("-" * 30)
        
        for param_name in idm_param_names:
            if param_name in full_data['full_time_idm_means']:
                full_mean = full_data['full_time_idm_means'][param_name]
                full_std = full_data['full_time_idm_stds'][param_name]
                
                window_param_means = []
                for window_result in window_data['window_results']:
                    if param_name in window_result['idm_means']:
                        window_param_means.append(window_result['idm_means'][param_name])
                
                if window_param_means:
                    window_mean = np.mean(window_param_means)
                    window_std = np.std(window_param_means)
                    temporal_variability = np.std(window_param_means)
                    
                    diff = abs(full_mean - window_mean)
                    diff_ratio = diff / full_mean if full_mean != 0 else diff
                    
                    print(f"  {param_name}:")
                    print(f"    Full-Time:  {full_mean:.3f} ± {full_std:.3f}")
                    print(f"    Window Avg: {window_mean:.3f} ± {window_std:.3f}")
                    print(f"    Difference: {diff:.3f} ({diff_ratio:.1%})")
                    print(f"    Temporal Variability: {temporal_variability:.3f}")

def perform_correlation_analysis(sliding_window_results):
    print("\n" + "="*80)
    print("PARAMETER CORRELATION ANALYSIS")
    print("="*80)
    
    all_correlations = []
    
    for vehicle_id, vehicle_data in sliding_window_results.items():
        driver_name = vehicle_data['driver_name']
        window_results = vehicle_data['window_results']
        
        if len(window_results) < 3:
            continue
        
        print(f"\n{driver_name} - Parameter Correlations:")
        print("-" * 40)
        
        param_data = {name: [] for name in idm_param_names + [f'rho_lag{lag+1}' for lag in range(d)]}
        
        for window in window_results:
            for param_name in idm_param_names:
                if param_name in window['idm_means']:
                    param_data[param_name].append(window['idm_means'][param_name])
            
            for lag in range(d):
                param_data[f'rho_lag{lag+1}'].append(window['ar_means'][lag])
        
        valid_params = [name for name in param_data.keys() if len(param_data[name]) == len(window_results)]
        
        if len(valid_params) > 1:
            correlation_matrix = np.zeros((len(valid_params), len(valid_params)))
            p_value_matrix = np.zeros((len(valid_params), len(valid_params)))
            
            for i, param1 in enumerate(valid_params):
                for j, param2 in enumerate(valid_params):
                    if i <= j:
                        corr, p_value = stats.pearsonr(param_data[param1], param_data[param2])
                        correlation_matrix[i, j] = corr
                        correlation_matrix[j, i] = corr
                        p_value_matrix[i, j] = p_value
                        p_value_matrix[j, i] = p_value
            
            print("Significant correlations (p < 0.05):")
            found_significant = False
            for i, param1 in enumerate(valid_params):
                for j, param2 in enumerate(valid_params):
                    if i < j and abs(correlation_matrix[i, j]) > 0.5 and p_value_matrix[i, j] < 0.05:
                        print(f"  {param1} vs {param2}: r = {correlation_matrix[i, j]:.3f} (p = {p_value_matrix[i, j]:.3f})")
                        found_significant = True
                        all_correlations.append({
                            'driver': driver_name,
                            'param1': param1,
                            'param2': param2,
                            'correlation': correlation_matrix[i, j],
                            'p_value': p_value_matrix[i, j]
                        })
            
            if not found_significant:
                print("  No significant correlations found")
        
        print(f"\n  Parameter values across windows:")
        for param_name in ['vmax', 'tsafe', 'amax', 'rho_lag1']:
            if param_name in param_data and len(param_data[param_name]) > 0:
                values = param_data[param_name]
                print(f"    {param_name}: {np.mean(values):.3f} ± {np.std(values):.3f} "
                      f"[{np.min(values):.3f}, {np.max(values):.3f}]")

def print_window_parameter_details(sliding_window_results):
    print("\n" + "="*80)
    print("DETAILED WINDOW PARAMETERS")
    print("="*80)
    
    for vehicle_id, vehicle_data in sliding_window_results.items():
        driver_name = vehicle_data['driver_name']
        
        print(f"\n{driver_name} - Window Parameters:")
        print("-" * 50)
        
        for i, window in enumerate(vehicle_data['window_results']):
            print(f"\nWindow {window['window_id']} (points: {window['n_obs']}):")
            
            ar_str = " | ".join([f"ρ_lag{lag+1}: {window['ar_means'][lag]:.3f}±{window['ar_stds'][lag]:.3f}" 
                               for lag in range(d)])
            print(f"  AR: {ar_str}")
            
            idm_params = []
            for param_name in ['vmax', 'tsafe', 'amax', 's_a', 's_v']:
                if param_name in window['idm_means']:
                    idm_params.append(f"{param_name}: {window['idm_means'][param_name]:.3f}±{window['idm_stds'][param_name]:.3f}")
            
            if idm_params:
                print(f"  IDM: {' | '.join(idm_params)}")

def plot_comparison(full_time_results, sliding_window_results):
    if not full_time_results or not sliding_window_results:
        print("No results to plot")
        return
    
    full_time_results_dict = full_time_results['full_time_results']
    
    print("\n" + "="*60)
    print("Generating Comparison Plots")
    print("="*60)
    
    for vehicle_id, full_data in full_time_results_dict.items():
        if vehicle_id not in sliding_window_results:
            continue
        
        window_data = sliding_window_results[vehicle_id]
        driver_name = full_data['driver_name']
        
        print(f"Plotting comparison for {driver_name}...")
        
        window_ar_means = np.array([result['ar_means'] for result in window_data['window_results']])
        window_ar_stds = np.array([result['ar_stds'] for result in window_data['window_results']])
        window_ids = [result['window_id'] for result in window_data['window_results']]
        
        full_time_means = full_data['full_time_ar_means']
        full_time_stds = full_data['full_time_ar_stds']
        
        for lag in range(d):
            fig, ax = plt.subplots(figsize=(12, 6))
            
            ax.plot(window_ids, window_ar_means[:, lag], 'o-', color='blue', 
                   linewidth=2, markersize=6, label='Sliding Window AR')
            ax.fill_between(window_ids, 
                           window_ar_means[:, lag] - window_ar_stds[:, lag], 
                           window_ar_means[:, lag] + window_ar_stds[:, lag], 
                           alpha=0.3, color='blue', label='Window Std')
            
            ax.axhline(y=full_time_means[lag], color='red', linestyle='--', 
                      linewidth=3, label='Full-Time AR Mean')
            
            ax.axhspan(full_time_means[lag] - full_time_stds[lag], 
                      full_time_means[lag] + full_time_stds[lag], 
                      alpha=0.2, color='red', label='Full-Time Std')
            
            ax.set_xlabel('Window ID', fontsize=12)
            ax.set_ylabel(f'AR Coefficient ρ (lag {lag+1})', fontsize=12)
            ax.set_title(f'{driver_name} - AR Coefficient Comparison (lag {lag+1})\n'
                        f'Full-Time: {full_time_means[lag]:.3f} ± {full_time_stds[lag]:.3f}', fontsize=14)
            ax.legend(fontsize=10)
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()

# Main execution
print("STEP 1: Full-time simultaneous calibration")
full_time_results = run_simultaneous_calibration()

print("\nSTEP 2: Sliding window analysis (corrected)")
sliding_window_results, window_population_params = run_sliding_window_analysis_corrected()

print("\nSTEP 3: Detailed parameter analysis")
if full_time_results and sliding_window_results:
    print_detailed_parameter_analysis(full_time_results, sliding_window_results)
    perform_correlation_analysis(sliding_window_results)
    print_window_parameter_details(sliding_window_results)
    
    print("\nSTEP 4: Generating comparison plots")
    plot_comparison(full_time_results, sliding_window_results)
else:
    print("Cannot perform analysis: missing calibration results")

print("\nAnalysis completed!")